In [1]:
import requests
import math
import json
import pandas as pd
from pymongo import MongoClient, GEO2D

In [2]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

ny = geocode("New York")
ny

{'type': 'Point', 'coordinates': [-73.9586, 40.68908]}

In [3]:
# https://docs.mongodb.com/manual/reference/operator/query/near/
def withGeoQuery(location,maxDistance=10000,minDistance=0,field="location"):
    return {
       field: {
         "$near": {
           "$geometry": location if type(location)==dict else geocode(location),
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }

In [4]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

## Filtamos por documentos que tengan los campos funding_round y offices no vacio, que no hayan quebrado

In [5]:
companies = list(db["companies"].find({"funding_rounds":{"$exists": True, "$not": {"$size": 0}},"offices":{"$exists": True, "$not": {"$size": 0}}, "offices.latitude":{"$exists":True, "$ne":None}, "offices.longitude":{"$exists":True, "$ne":None},"deadpooled_year":{"$not":{"$gte":0}}},{"name":1,"offices":1,"category_code":1, "funding_rounds":1}))
df = pd.DataFrame(companies)
df.head()

,_id,name,category_code,funding_rounds,offices
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis..."
1,52cdef7c4bab8bd675297d91,Geni,web,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d96,Gizmoz,web,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr..."
3,52cdef7c4bab8bd675297d97,Scribd,news,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant..."
4,52cdef7c4bab8bd675297d99,Lala,games_video,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address..."


In [6]:
def sum_funding_rounds(funding_rounds):
    currency_dict = {"USD":1, "EUR":1.09, "GBP":1.29, "CAD":0.75, "SEK":0.1, "JPY":0.0091}
    result = []
    for e in df.funding_rounds:
        result.append(sum([0 if k["raised_amount"] == None else k["raised_amount"]*currency_dict[k["raised_currency_code"]] for k in e if "raised_amount" in k.keys() and "raised_currency_code" in k.keys()]))
    return result

In [7]:
total_funding = sum_funding_rounds(df.funding_rounds)

In [8]:
df["Total_funding"] = total_funding

In [9]:
df.head()

,_id,name,category_code,funding_rounds,offices,Total_funding
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis...",45000000.0
1,52cdef7c4bab8bd675297d91,Geni,web,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '...",16500000.0
2,52cdef7c4bab8bd675297d96,Gizmoz,web,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr...",18100000.0
3,52cdef7c4bab8bd675297d97,Scribd,news,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant...",25762000.0
4,52cdef7c4bab8bd675297d99,Lala,games_video,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address...",44150000.0


In [10]:
df_clean = df[df.Total_funding > 1000000]
df_clean

,_id,name,category_code,funding_rounds,offices,Total_funding
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis...",45000000.0
1,52cdef7c4bab8bd675297d91,Geni,web,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '...",16500000.0
2,52cdef7c4bab8bd675297d96,Gizmoz,web,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr...",18100000.0
3,52cdef7c4bab8bd675297d97,Scribd,news,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant...",25762000.0
4,52cdef7c4bab8bd675297d99,Lala,games_video,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address...",44150000.0
...,...,...,...,...,...,...
2975,52cdef7f4bab8bd67529c6dd,Cameron Health,medical,"[{'id': 7368, 'round_code': 'e', 'source_url':...","[{'description': 'Corporate Office', 'address1...",121000000.0
2976,52cdef7f4bab8bd67529c6e3,InSound Medical,biotech,"[{'id': 7373, 'round_code': 'e', 'source_url':...","[{'description': '', 'address1': '39660 Eureka...",11000000.0
2977,52cdef7f4bab8bd67529c6de,Cantimer,None,"[{'id': 7369, 'round_code': 'a', 'source_url':...","[{'description': '', 'address1': '3700 Haven C...",2000000.0
2978,52cdef7f4bab8bd67529c6e1,Purfresh,cleantech,"[{'id': 8564, 'round_code': 'b', 'source_url':...","[{'description': 'HQ', 'address1': '1350 Willo...",62700000.0


In [11]:
df_clean = df_clean.explode('offices')
df_offices= df_clean[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
df_clean = pd.concat([df_clean,df_offices], axis=1)
df_clean.head(10)

,_id,name,category_code,funding_rounds,offices,Total_funding,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8d,Digg,news,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","{'description': None, 'address1': '135 Mississ...",4.500000e+07,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
1,52cdef7c4bab8bd675297d91,Geni,web,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","{'description': 'Headquarters', 'address1': '9...",1.650000e+07,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d96,Gizmoz,web,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","{'description': None, 'address1': None, 'addre...",1.810000e+07,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472
3,52cdef7c4bab8bd675297d97,Scribd,news,"[{'id': 10, 'round_code': 'seed', 'source_url'...","{'description': 'HQ', 'address1': '539 Bryant ...",2.576200e+07,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
4,52cdef7c4bab8bd675297d99,Lala,games_video,"[{'id': 16, 'round_code': 'b', 'source_url': '...","{'description': 'Lala Headquarters', 'address1...",4.415000e+07,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
5,52cdef7c4bab8bd675297d98,Slacker,music,"[{'id': 12, 'round_code': 'b', 'source_url': '...","{'description': '', 'address1': '16935 W. Bern...",7.310000e+07,,16935 W. Bernardo Dr. Suite 101,,92127,San Diego,CA,USA,33.022176,-117.081406
6,52cdef7c4bab8bd675297d9c,MeetMoi,social,"[{'id': 18, 'round_code': 'a', 'source_url': '...","{'description': None, 'address1': '', 'address...",5.575000e+06,None,,,,New York City,NY,USA,40.757929,-73.985506
7,52cdef7c4bab8bd675297d9d,Joost,games_video,"[{'id': 19, 'round_code': 'a', 'source_url': '...","{'description': '', 'address1': '100 5th Ave F...",4.500000e+07,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447
8,52cdef7c4bab8bd675297d94,Twitter,social,"[{'id': 261, 'round_code': 'a', 'source_url': ...","{'description': '', 'address1': '1355 Market S...",1.160167e+09,,1355 Market St.,,94103,San Francisco,CA,USA,37.776805,-122.416924
9,52cdef7c4bab8bd675297da1,Plaxo,web,"[{'id': 20, 'round_code': 'a', 'source_url': '...","{'description': 'HQ', 'address1': '1050 Enterp...",2.830000e+07,HQ,"1050 Enterprise Way, 5th floor",,94089,Sunnyvale,CA,USA,37.387845,-122.055197


In [12]:
df_clean = df_clean.drop(columns = ["offices","funding_rounds"])

In [13]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None

In [14]:
df_clean["location"] = df_clean[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
df_clean = df_clean.drop(columns=["latitude","longitude","_id","address1", "address2","zip_code"])

In [15]:
df_clean

,name,category_code,Total_funding,description,city,state_code,country_code,location
0,Digg,news,45000000.0,None,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Geni,web,16500000.0,Headquarters,West Hollywood,CA,USA,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,Gizmoz,web,18100000.0,None,Menlo Park,CA,USA,"{'type': 'Point', 'coordinates': [-122.169472,..."
3,Scribd,news,25762000.0,HQ,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.404052,..."
4,Lala,games_video,44150000.0,Lala Headquarters,Palo Alto,CA,USA,"{'type': 'Point', 'coordinates': [-122.154369,..."
...,...,...,...,...,...,...,...,...
2975,Cameron Health,medical,121000000.0,Corporate Office,San Clemente,CA,USA,"{'type': 'Point', 'coordinates': [-117.607717,..."
2976,InSound Medical,biotech,11000000.0,,Newark,CA,USA,"{'type': 'Point', 'coordinates': [-121.998513,..."
2977,Cantimer,None,2000000.0,,Menlo Park,CA,USA,"{'type': 'Point', 'coordinates': [-122.181753,..."
2978,Purfresh,cleantech,62700000.0,HQ,Menlo Park,CA,USA,"{'type': 'Point', 'coordinates': [-122.151599,..."


In [16]:
df_clean.to_json("OUTPUT/cleaned_offices.json", orient="records")

In [20]:
client = MongoClient("mongodb://localhost/")
db = client["companies"]
companies_clean = db["companies_clean"]
with open('OUTPUT/cleaned_offices.json') as f:
    file_data = json.load(f)
companies_clean.insert_many(file_data)
companies_clean.create_index([("location", "2dsphere")])
client.close()

In [ ]:
db.places.create_index([("location", GEO2D)])